In [4]:
# === hdbscan_minimal_with_fallback_v2.py ===
import os, json, pickle, numpy as np, pandas as pd
from itertools import product
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.neighbors import NearestNeighbors
import hdbscan
from hdbscan import all_points_membership_vectors

# ------- 轻量可调项 -------
DATA_PATH        = 'data/fashion/handled/pca64_itm_emb_np.pkl'
OUT_DIR          = './artifacts_hdbscan_min'
SEED             = 42
EVAL_SUBSAMPLE   = 8000    # 评价抽样上限（提速）
MAX_NOISE_RATIO  = 0.5     # 搜索过滤：噪声比例上限
MIN_N_CLUSTERS   = 2       # 略放松，便于起步（原为 3）
USE_STANDARDIZE  = False   # ★ 可切 True 看看是否更稳（会打印“数据范围”）
min_cluster_size_list = [3, 5, 8, 10, 12, 15, 20, 25, 30, 40, 50]
min_samples_list      = [None, 1, 2, 3, 5, 8, 10]
# -------------------------

os.makedirs(OUT_DIR, exist_ok=True)
rng = np.random.default_rng(SEED)

def load_embeddings(path):
    with open(path, 'rb') as f:
        X = pickle.load(f)
    print(f"[Load] {X.shape}")
    return X

def prep_features(X_raw, use_standardize=USE_STANDARDIZE):
    # 可选：先标准化，再 L2
    if use_standardize:
        scaler = StandardScaler(with_mean=True, with_std=True)
        Xs = scaler.fit_transform(X_raw)
        print(f"[Scale] range after StandardScaler: min={Xs.min():.3f}, max={Xs.max():.3f}")
        X = normalize(Xs, norm='l2', axis=1)
    else:
        X = normalize(X_raw, norm='l2', axis=1)
    # 简要检查
    rn = np.linalg.norm(X, axis=1)
    print(f"[Check] row-norm mean/min/max = {rn.mean():.4f} / {rn.min():.4f} / {rn.max():.4f}")
    # KNN 健康检查（密度差异是否存在）
    nbrs = NearestNeighbors(n_neighbors=10, metric='cosine').fit(X)
    dists, _ = nbrs.kneighbors(X)
    p50 = np.percentile(dists[:, -1], 50)
    p90 = np.percentile(dists[:, -1], 90)
    print(f"[KNN-10] 第10近邻距离 p50={p50:.4f}  p90={p90:.4f}")
    return X

def hdbscan_clustering(X, min_cluster_size=25, min_samples=None, metric='cosine', eval_subsample=EVAL_SUBSAMPLE):
    model = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric=metric,
        cluster_selection_method='eom',
        prediction_data=True,
        core_dist_n_jobs=os.cpu_count(),
        allow_single_cluster=True,       # ★ 新增：允许得到单簇，避免全噪声
        cluster_selection_epsilon=0.05   # ★ 新增：稍放松簇选择阈值
    ).fit(X)
    labels = model.labels_.astype(np.int32)
    probs  = model.probabilities_.astype(np.float32)

    # 统计
    uniq, cnts = np.unique(labels, return_counts=True)
    n_clusters  = len(uniq) - (1 if -1 in uniq else 0)
    noise_count = cnts[uniq == -1][0] if -1 in uniq else 0
    noise_ratio = float(noise_count) / len(X)

    # 指标（排噪声 + 抽样）
    sil, cal = None, None
    if n_clusters > 1:
        idx = np.where(labels != -1)[0]
        if idx.size >= 3 and np.unique(labels[idx]).size > 1:
            if idx.size > eval_subsample:
                idx = rng.choice(idx, size=eval_subsample, replace=False)
            X_eval, y_eval = X[idx], labels[idx]
            try: sil = float(silhouette_score(X_eval, y_eval, metric=metric))
            except: sil = None
            try: cal = float(calinski_harabasz_score(X_eval, y_eval))
            except: cal = None

    metrics = dict(n_clusters=int(n_clusters), noise_ratio=noise_ratio, silhouette=sil, calinski=cal)
    return labels, probs, model, metrics

def grid_search(X, mcs_list, ms_list, metric='cosine'):
    rows = []
    for mcs, ms in product(mcs_list, ms_list):
        # 经验性约束：过大 min_samples 容易全噪声
        if ms is not None and ms > mcs:
            continue
        try:
            labels, probs, _, m = hdbscan_clustering(X, mcs, ms, metric)
            rows.append(dict(
                min_cluster_size=int(mcs),
                min_samples=(None if ms is None else int(ms)),
                N=m['n_clusters'],
                noise_ratio=m['noise_ratio'],
                silhouette=m['silhouette'],
                calinski=m['calinski'],
                avg_prob=float(np.mean(probs)) if probs.size else np.nan
            ))
        except Exception:
            rows.append(dict(
                min_cluster_size=int(mcs), min_samples=(None if ms is None else int(ms)),
                N=np.nan, noise_ratio=1.0, silhouette=np.nan, calinski=np.nan, avg_prob=np.nan
            ))
    df = pd.DataFrame(rows)

    # 过滤
    valid = df[(df['noise_ratio'] <= MAX_NOISE_RATIO) & (df['N'] >= MIN_N_CLUSTERS)].copy()
    if valid.empty:
        print("[Warn] 无满足约束的组合，放宽为全部结果的排序参考。")
        valid = df.copy()

    # 简单综合评分：sil 主导 + 低噪声 + 高 avg_prob，少量鼓励更多簇
    def score(r):
        s = 0.0
        if pd.notna(r['silhouette']): s += 1.0 * r['silhouette']
        if pd.notna(r['avg_prob']):   s += 0.2 * r['avg_prob']
        s += 0.05 * np.log1p(max(r['N'], 0))
        s -= 0.5 * r['noise_ratio']
        return s

    valid['score'] = valid.apply(score, axis=1)
    valid = valid.sort_values(by='score', ascending=False).reset_index(drop=True)
    return valid, df

# ---------- 主流程 ----------
X_raw = load_embeddings(DATA_PATH)
X = prep_features(X_raw, USE_STANDARDIZE)

# A) 先在原空间试 Euclidean（很多数据在欧氏下更易分簇）
print("\n=== 先试欧氏距离（原空间/L2）===")
leaderboard_eu, raw_grid_eu = grid_search(X, min_cluster_size_list, min_samples_list, metric='euclidean')
direct_eu_ok = not leaderboard_eu['N'].isna().all() and not (leaderboard_eu['N'].fillna(0) == 0).all()

if direct_eu_ok:
    leaderboard, raw_grid = leaderboard_eu, raw_grid_eu
    DIRECT_EUCLIDEAN = True
else:
    DIRECT_EUCLIDEAN = False
    print("\n=== 再试 Cosine（原空间/L2）===")
    leaderboard, raw_grid = grid_search(X, min_cluster_size_list, min_samples_list, metric='cosine')

print(leaderboard.head(10))

# B) 若原空间两种度量都不行，走 UMAP 兜底
need_fallback = leaderboard['N'].isna().all() or (leaderboard['N'].fillna(0) == 0).all()
if need_fallback:
    print("\n[Fallback] 启用 UMAP(10维, cosine) + HDBSCAN(euclidean) 兜底...")
    import umap
    reducer = umap.UMAP(
        n_components=10,     # ★ 更高一点的低维空间
        n_neighbors=30,      # ★ 扩大邻域，增强全局结构
        min_dist=0.0,        # ★ 拉开簇间距
        random_state=SEED,
        metric='cosine'
    )
    X_umap = reducer.fit_transform(X)
    # 在低维欧式空间再搜一次（更宽松）
    mcs_list_fb = [3, 5, 8, 10, 12, 15, 20]
    ms_list_fb  = [None, 1, 2, 3, 5]
    leaderboard, raw_grid = grid_search(X_umap, mcs_list_fb, ms_list_fb, metric='euclidean')
    print(leaderboard.head(10))
    if leaderboard.empty:
        raise RuntimeError("UMAP 兜底后仍无可用最优参数，请进一步放宽搜索/检查数据。")
    # 后续流程切换到降维后的 X
    X = X_umap

if leaderboard.empty:
    raise RuntimeError("无可用最优参数。请放宽筛选条件或调整搜索空间。")

best = leaderboard.iloc[0]
# 根据实际走通的路径确定 metric
if need_fallback:
    chosen_metric = 'euclidean'
elif 'DIRECT_EUCLIDEAN' in globals() and DIRECT_EUCLIDEAN:
    chosen_metric = 'euclidean'
else:
    chosen_metric = 'cosine'

best_params = dict(
    min_cluster_size=int(best['min_cluster_size']),
    min_samples=(None if pd.isna(best['min_samples']) else int(best['min_samples'])),
    metric=chosen_metric,
    cluster_selection_method='eom'
)
print("\n=== 最优参数 ===")
print(best_params)

labels, probs, model, metrics = hdbscan_clustering(
    X,
    min_cluster_size=best_params['min_cluster_size'],
    min_samples=best_params['min_samples'],
    metric=best_params['metric']
)

# 加权簇中心（权重=probabilities_），单位化（配合 cosine；若是欧氏也无害）
centers, cid_list = [], []
for cid in sorted(c for c in np.unique(labels) if c != -1):
    idx = np.where(labels == cid)[0]
    if idx.size == 0: 
        continue
    w = probs[idx].reshape(-1, 1).astype(np.float32)
    w = np.nan_to_num(w, nan=0.0, posinf=0.0, neginf=0.0)
    s = np.sum(w)
    if s <= 1e-12:
        c = X[idx].mean(axis=0)
    else:
        c = (X[idx] * w).sum(axis=0) / s
    c = c / (np.linalg.norm(c) + 1e-12)
    centers.append(c.astype(np.float32))
    cid_list.append(int(cid))
centers = np.vstack(centers) if len(centers) else np.zeros((0, X.shape[1]), dtype=np.float32)
center_cluster_ids = np.array(cid_list, dtype=np.int32)

# 全簇 membership 矩阵（FDAC 直接用）
U = all_points_membership_vectors(model).astype(np.float32)

# ---------- 落盘（最小必需集） ----------
np.save(os.path.join(OUT_DIR, "labels.npy"), labels.astype(np.int32))
np.save(os.path.join(OUT_DIR, "probs.npy"),  probs.astype(np.float32))
np.save(os.path.join(OUT_DIR, "centers.npy"), centers.astype(np.float32))
np.save(os.path.join(OUT_DIR, "center_cluster_ids.npy"), center_cluster_ids)
np.save(os.path.join(OUT_DIR, "memberships.npy"), U.astype(np.float32))

with open(os.path.join(OUT_DIR, "best_params.json"), "w", encoding="utf-8") as f:
    json.dump(best_params, f, ensure_ascii=False, indent=2)
with open(os.path.join(OUT_DIR, "metrics.json"), "w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)

print(f"\n[保存完成] {OUT_DIR}")
print("  - labels.npy / probs.npy / centers.npy / center_cluster_ids.npy / memberships.npy")
print("  - best_params.json / metrics.json")


[Load] (4722, 64)
[Check] row-norm mean/min/max = 1.0000 / 1.0000 / 1.0000
[KNN-10] 第10近邻距离 p50=0.3674  p90=0.4805

=== 先试欧氏距离（原空间/L2）===


/root/miniconda3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.w

KeyboardInterrupt: 